In [2]:
# install what needed
!pip install pandas python-docx
!pip install pywin32

In [76]:
import pandas as pd
import win32com.client as win32
from datetime import datetime
import os

# Load Excel data
MyFile = "April2024.xlsx" # input("Enter your file path: ")
df = pd.read_excel(MyFile)

# Start Word
word = win32.gencache.EnsureDispatch('Word.Application')
word.Visible = True

MonthsNames = ['ינואר','פברואר','מרץ','אפריל','מאי','יוני','יולי','אוגוסט','ספטמבר','אוקטובר','נובמבר','דצמבר']

# Loop over each row in the Excel data
FirstMonth = 4
Length_of_Month_Vec = 13-FirstMonth
FirstPayer = 1
LastPayer = 34
First_Day_to_Look = 11.4 #only new payer

Payers = range(2*(FirstPayer-1), 2*LastPayer-1, 2) #(np.array([4,12,15,25])-1)*2 #
for i in Payers:  # size(raw, 1)
    # collect date
    Name = df.iloc[i, 0]
    Apartment = int(df.iloc[i, 1])
    Methodi = df.iloc[i+1, 2]
    if pd.isnull(Methodi):
        Methodi = 'העברה'
    paymentsVec = df.iloc[i, 3:15].values
    payments = paymentsVec[~pd.isnull(paymentsVec)]
    Sum = sum(payments)
    paidMonths = round(Sum/250)
    if paidMonths==0:
        continue
    
    DateVector = [datetime.strptime(x+'.24', 'שולם %d.%m.%y') if isinstance(x, str) else None for x in df.iloc[i+1, 3:15]]
    ValiDate = [x is not None for x in DateVector]

    PaidVec = [False]*12
    PaidVec[:paidMonths] = [True]*paidMonths
    PaidMonthVec = [0]*12
    Last_i = 0
    for Vi in range(1,len(ValiDate)+1):
        if ValiDate[Vi-1]:
            nowNumMonth = round(paymentsVec[Vi-1]/250)
            if nowNumMonth > 0:
                PaidMonthVec[Last_i:Last_i+nowNumMonth] = [Vi]*(nowNumMonth)
                Last_i = Last_i + nowNumMonth
            else:
                Last_i = Last_i + 1

    # Fixing Month in First day to look
    if (First_Day_to_Look%1)==0:
        First_Day_to_Look +=  float("0."+str(FirstMonth))
    
    is_after_date = [DateVector[date_index]>=datetime.strptime(str(First_Day_to_Look)+".24",'%d.%m.%y') if DateVector[date_index] is not None else False for date_index in range(0,len(DateVector),1)]
    # if the start date is after all payments, continue next payer
    if not any(is_after_date[FirstMonth-1:]):
        continue
    
    RelevantPaymentVec = payments[FirstMonth-1:]
    RelevantPaid = RelevantPaymentVec[~pd.isnull(RelevantPaymentVec)]
    RelevantSum  = sum(RelevantPaid)
    if RelevantSum<1:
        continue
    RelevandPaidMonths = round(RelevantSum/250) 
    monthStr = '{:02d}'.format(RelevandPaidMonths)
    monthNumStr = '{:02d}'.format(max([min([PaidMonthVec.index(x)+1 for x in PaidMonthVec if x != 0]),FirstMonth]))
    yearStr = DateVector[ValiDate.index(True)].strftime('%y')
    receiptNumber = '{:02d}'.format(Apartment) + monthStr + monthNumStr + yearStr

    # Open the template document
    doc = word.Documents.Open(os.path.join(os.getcwd(), 'template.docx'))

    # Save the new document
    doc.SaveAs(os.path.join(os.getcwd(), 'April2\קבלה ' + receiptNumber + ' ועד בית.docx'))

    # Replace placeholders with data from Excel
    word.Selection.Find.Execute('{Name}', False, False, False, False, False, True, 1, True, Name, 2)
    word.Selection.Find.Execute('{Apartment}', False, False, False, False, False, True, 1, True, str(Apartment), 2)
    word.Selection.Find.Execute('{Date}', False, False, False, False, False, True, 1, True, datetime.now().strftime('%d/%m/%Y'), 2)
    word.Selection.Find.Execute('{Receipt}', False, False, False, False, False, True, 1, True, receiptNumber, 2)

    # Get the existing table
    table = doc.Tables(1)

    # Fill table with data
    index = 0
    for Vi in range(FirstMonth, len(ValiDate)+1):
        if ValiDate[Vi-1]:
            try:
                AllNames = [MonthsNames[ind] for ind,x in enumerate(PaidMonthVec) if x == Vi]
                monthString = ' - '.join(list(set([AllNames[0], AllNames[-1]])))
            except:
                continue
            index = index + 1
            row = table.Rows.Add()
            row.Cells(1).Range.Text = str(index)
            row.Cells(2).Range.Text = 'תשלום עבור ' + monthString + ' 2024'
            row.Cells(3).Range.Text = Methodi
            row.Cells(4).Range.Text = DateVector[Vi-1].strftime('%d/%m/%y')
            row.Cells(5).Range.Text = str(paymentsVec[Vi-1]) + ' ₪'

    # Replace {Sum} placeholder with sum of amounts
    word.Selection.Find.Execute('{Sum}', False, False, False, False, False, True, 1, True, str(RelevantSum), 2)

    doc.Save()
    doc.SaveAs(os.path.join(os.getcwd(), 'April2\קבלה ' + receiptNumber + ' ועד בית.pdf'), FileFormat=17)
    doc.Close()

word.Quit()


In [28]:
First_Day_to_Look = 1.1
[DateVector[date_index]>=datetime.strptime(str(First_Day_to_Look)+".24",'%d.%m.%y') if DateVector[date_index] is not None else False for date_index in range(0,len(DateVector),1)]

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [24]:
First_Day_to_Look = 1.4
is_after_time = [False]*12
for date_index in range(0,len(DateVector),1):
    if DateVector[date_index] is not None:
        is_after_time[date_index] = DateVector[date_index]>=datetime.strptime(str(First_Day_to_Look)+".24",'%d.%m.%y')
    else:
        is_after_time[date_index] = False

print(is_after_time)

[False, False, False, False, False, False, False, False, False, False, False, False]


In [46]:
print(is_after_time)
print(not any(is_after_time[4:]))


[True, True, True, True, False, False, False, False, False, False, False, False, False]
True


In [62]:
is_after_time

[True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [72]:
max([min([PaidMonthVec.index(x)+1 for x in PaidMonthVec if x != 0]),FirstMonth])

4

In [71]:
FirstMonth = 4

FirstMonth

4

In [73]:
RelevantPaymentVec

array([250, nan, nan, nan, nan, nan, nan, nan, nan], dtype=object)